In [1]:
!pip install datasets

In [2]:
import kagglehub
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import T5Tokenizer
from datasets import load_dataset
import re
import string
from nltk.corpus import stopwords
import nltk

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
validation_df = pd.read_csv("/kaggle/input/twitter-airline-sentiment/Tweets.csv")

In [5]:
validation_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [6]:
X=validation_df["text"].tolist()
y=validation_df["airline_sentiment"].tolist()
train_X, temp_X, train_y, temp_y = train_test_split(X, y, test_size=0.3, random_state=42)
val_X, test_X, val_y, test_y = train_test_split(temp_X, temp_y, test_size=0.5, random_state=42)

In [7]:
type(train_X)

list

In [8]:
class SummaryDataset(Dataset):
    def __init__(self, inputs, labels, tokenizer, encoder_maxlen): 
        self.inputs = inputs
        self.labels = labels  # Assign the labels to self.labels
        self.tokenizer = tokenizer
        self.encoder_maxlen = encoder_maxlen

        # Create label mapping
        self.label_map = {'positive': 0, 'negative': 1, 'neutral': 2}  # Define your labels
        
        # Convert string labels to numerical labels
        self.numerical_labels = [self.label_map[label] if label in self.label_map else -1 for label in self.labels]  # Handle unknown labels if needed

    def preprocess_text(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s.,!?%$&@#:]', '', text)
        text = ' '.join(text.split())
        return text

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.preprocess_text(str(self.inputs[idx]))
        label_id = self.numerical_labels[idx]  # Get numerical label

        input_encodings = self.tokenizer(
            input_text,
            max_length=self.encoder_maxlen,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return (
            input_encodings['input_ids'].squeeze(0),
            input_encodings['attention_mask'].squeeze(0),
            torch.tensor(label_id)  # Return numerical label as tensor
        )

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoder_maxlen = 768

train_dataset = SummaryDataset(train_X, train_y, tokenizer, encoder_maxlen)
val_dataset = SummaryDataset(val_X, val_y, tokenizer, encoder_maxlen)
test_dataset = SummaryDataset(test_X, test_y, tokenizer, encoder_maxlen)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
def angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates
def positional_encoding(position, d_model):
    angle_rads = angles(np.arange(position)[:, np.newaxis],
                       np.arange(d_model)[np.newaxis, :],
                       d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return torch.FloatTensor(pos_encoding).to(device)

In [10]:
def create_padding_mask(seq):
    mask = (seq == 0).float()
    return mask[:, None, None, :]

def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones((size, size)), diagonal=1)
    return mask

In [11]:
def scaled_dot_product_attention(q, k, v, mask=None):
  q, k, v = q.to(device), k.to(device), v.to(device)
  depth = q.size(-1)
  dots = torch.matmul(q, k.transpose(-2, -1))

  if mask is not None:
      dots = dots.masked_fill(mask.bool(), -1e9)
  attention_weights = F.softmax(dots, dim=-1)
  output = torch.matmul(attention_weights, v)

  return output, attention_weights
def pointwise_ffn(d_model, dff):
  return torch.nn.Sequential(
      torch.nn.Linear(d_model, dff),
      torch.nn.ReLU(),
      torch.nn.Linear(dff, d_model)
  ).to(device)

In [12]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads
    #Linear layer for q k v
    self.wq = nn.Linear(d_model, d_model)
    self.wk = nn.Linear(d_model, d_model)
    self.wv = nn.Linear(d_model, d_model)
    # Final linear layer
    self.dense = nn.Linear(d_model, d_model)
  def split_heads(self, x, batch_size):  # Added split_heads function
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)
  def forward(self, v, k, q, mask=None):
        batch_size = q.size(0)
        # Apply linear transformations to queries, keys, and values
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        # Split the heads for multi-head attention
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        # Compute scaled dot-product attention
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # Concatenate the attention outputs
        scaled_attention = scaled_attention.permute(0, 2, 1, 3)
        concat_attention = scaled_attention.contiguous().view(batch_size, -1, self.d_model)

        # Pass through the final dense layer
        output = self.dense(concat_attention)

        return output, attention_weights

In [13]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dff, rate=0.2):
    super(EncoderLayer, self).__init__()
    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = pointwise_ffn(d_model, dff)

    self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6)
    self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6)

    self.dropout1 = nn.Dropout(rate)
    self.dropout2 = nn.Dropout(rate)

  def forward(self, x, training, mask):
    attention_output, _ = self.mha(x, x, x, mask)
    attention_output = self.dropout1(attention_output)
    out1 = self.layernorm1(x + attention_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

In [14]:
class Encoder(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.2):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = nn.Embedding(input_vocab_size, d_model)


    self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)
    self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

    self.dropout = nn.Dropout(rate)

  def forward(self, x, training, mask):
    seq_len = x.shape[1]
    x = self.embedding(x)
    x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)) #d=d
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x)
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    return x

In [15]:
class Transformer(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, num_classes, pe_input, rate=0.2,
               hidden_layer_size=256, classifier_dropout_rate=0.3):
    super(Transformer, self).__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

    self.mlp_classifier = nn.Sequential(
        nn.Linear(d_model, hidden_layer_size),
        nn.ReLU(),
        nn.Dropout(classifier_dropout_rate),
        nn.Linear(hidden_layer_size, num_classes)
    )

  def forward(self, inp, training, enc_padding_mask):
    encoder_output = self.encoder(inp, training, enc_padding_mask)
    sequence_pooled_output = torch.mean(encoder_output, dim=1)
    mlp_output = self.mlp_classifier(sequence_pooled_output)
    final_output = mlp_output
    return final_output

In [16]:
tokenizer.vocab_size

30522

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
transformer = Transformer(
    num_layers=6,
    d_model=512,
    num_heads=8,
    dff=2048,
    input_vocab_size=tokenizer.vocab_size,
    num_classes=len(train_dataset.label_map), # Use num_classes based on your dataset
    pe_input=768,
).to(device)

In [19]:
# Print model architecture
print("Model Architecture:")
print(transformer)

# Print number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'\nNumber of trainable parameters: {count_parameters(transformer)}')
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
epochs = 20
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

loss = nn.CrossEntropyLoss()
optimizer = AdamW(transformer.parameters(), lr=2e-5 ,betas=(0.9, 0.999), eps=1e-9, weight_decay = 0.01)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=len(train_dataloader) * 2, 
    num_training_steps=len(train_dataloader) * epochs
)

Model Architecture:
Transformer(
  (encoder): Encoder(
    (embedding): Embedding(30522, 512)
    (enc_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (mha): MultiHeadAttention(
          (wq): Linear(in_features=512, out_features=512, bias=True)
          (wk): Linear(in_features=512, out_features=512, bias=True)
          (wv): Linear(in_features=512, out_features=512, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (ffn): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (layernorm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (layernorm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (mlp_cl

In [ ]:
def loss_func(real, pred):
    # Reshape predictions
    pred = pred.contiguous().view(-1, pred.size(-1))
    real = real.contiguous().view(-1)

    # Calculate loss
    loss = F.cross_entropy(
        pred,
        real,
        reduction='mean'
    )
    return loss


In [ ]:

checkpoint_path = '/kaggle/working/checkpoint.pth'
def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path = '/kaggle/working/checkpoint.pth'):
    """Save model and optimizer state as a checkpoint."""
    checkpoint = {
        # 'epoch': epoch,
        'model_state_dict': model.state_dict(),
        # 'optimizer_state_dict': optimizer.state_dict(),
        # 'loss': loss,
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch}, loss: {loss:.4f}")

In [ ]:
def create_masks(input_ids, input_mask):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(input_ids)

    return enc_padding_mask


In [ ]:
def train_step(input_ids, input_mask, target_ids):

    enc_padding_mask = create_masks(input_ids, input_mask)

    optimizer.zero_grad()
    predictions = transformer(input_ids, True,
                               enc_padding_mask)

    loss = loss_func(target_ids, predictions)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(transformer.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()

    return loss.item()

In [ ]:
def evaluate(val_dataset, batch_size=32):
    transformer.eval()
    total_loss = 0
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, input_mask, target_ids = [x.to(device) for x in batch]

            enc_padding_mask = create_masks(input_ids, input_mask)

            predictions = transformer(input_ids, False,
                                      enc_padding_mask)

            loss = loss_func(target_ids, predictions)
            total_loss += loss.item()

    return total_loss / len(val_dataloader)

In [20]:



# Training loop
batch_size = 32
epochs = 20
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

best_val_loss = float('inf')
patience = 2 # You can adjust patience value
trigger_times = 0

for epoch in range(epochs):
    transformer.train()
    total_loss = 0
    batch_count = 0

    print(f'\nEpoch {epoch + 1}/{epochs}')
    print('-' * 40)

    for batch_idx, batch in enumerate(train_dataloader):
        input_ids, input_mask, target_ids = [x.to(device) for x in batch]
        batch_loss = train_step(input_ids, input_mask, target_ids)
        total_loss += batch_loss
        batch_count += 1
        # Print progress every 100 batches
        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(train_dataloader)}, Loss: {batch_loss:.4f}')

    # Calculate average loss for the epoch
    avg_train_loss = total_loss / batch_count

    # Validation at the end of each epoch
    val_loss = evaluate(val_dataset)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(transformer.state_dict(), 'best_model.pth')
        print("save there")
    else:
        trigger_times += 1
        print(f"No improvement in validation loss for {trigger_times} epoch(s)")

        if trigger_times >= patience:
            print(f"Early stopping triggered after {patience} epochs without improvement.")
            break # Break out of training loop

    # Save checkpoint
    save_checkpoint(transformer, optimizer, epoch + 1, avg_train_loss)
    # Print epoch summary
    print('\nEpoch Summary:')
    print(f'Average Training Loss: {avg_train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')
    print(f'Learning Rate: {scheduler.get_last_lr()[0]:.2e}')


Epoch 1/20
----------------------------------------
Batch 0/321, Loss: 1.1444
Batch 100/321, Loss: 0.9358
Batch 200/321, Loss: 0.9306
Batch 300/321, Loss: 0.8397
save there
Checkpoint saved at epoch 1, loss: 0.9156

Epoch Summary:
Average Training Loss: 0.9156
Validation Loss: 0.7491
Learning Rate: 1.00e-05

Epoch 2/20
----------------------------------------
Batch 0/321, Loss: 0.9637
Batch 100/321, Loss: 0.5115
Batch 200/321, Loss: 0.5625
Batch 300/321, Loss: 0.7318
save there
Checkpoint saved at epoch 2, loss: 0.7123

Epoch Summary:
Average Training Loss: 0.7123
Validation Loss: 0.6605
Learning Rate: 2.00e-05

Epoch 3/20
----------------------------------------
Batch 0/321, Loss: 0.5021
Batch 100/321, Loss: 0.4945
Batch 200/321, Loss: 0.5351
Batch 300/321, Loss: 0.4638
save there
Checkpoint saved at epoch 3, loss: 0.6237

Epoch Summary:
Average Training Loss: 0.6237
Validation Loss: 0.6043
Learning Rate: 1.89e-05

Epoch 4/20
----------------------------------------
Batch 0/321, Loss

In [23]:


def evaluate_accuracy(test_dataset, tokenizer, model, device, label_map):
    """Calculates the accuracy on the test set."""
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    for input_ids, input_mask, true_label in DataLoader(test_dataset, batch_size=1, shuffle=False): # batch size of 1
        
        text = tokenizer.decode(input_ids.squeeze(0), skip_special_tokens=True) # decode the input

        predicted_label = predict_sentiment(text, tokenizer, model, device, label_map)
        true_label_str = {v:k for k, v in label_map.items()}[true_label.item()]


        if predicted_label == true_label_str:
          correct_predictions += 1
        total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate the accuracy
accuracy = evaluate_accuracy(test_dataset, tokenizer, transformer, device, train_dataset.label_map)

print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8097
